In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
 
from tools import *

import os
import warnings
warnings.filterwarnings('ignore')

import gc
import joblib

# Random Forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GroupShuffleSplit, cross_val_score

In [5]:
CACHE_PATH = '/Users/angus/Downloads/kaggle/'
cpath = lambda x: CACHE_PATH + x

In [93]:
df = pd.read_pickle(cpath('base_data'))
df_structures = pd.read_csv('../input/structures.csv');

In [94]:
df = get_portion(df, r=0.1)
df.shape

(461749, 23)

In [95]:
df = encode_cols(df, ['type'])

```
feats = ['n_H', 'y_1', 'atom_1_max_dist', 'atom_0_mean_dist', 'atom_1_med_dist',
       'dist', 'x_0', 'z_1', 'atom_0_min_dist', 'atom_0_med_dist',
       'atom_0_max_dist', 'nearby_C', 'n_N', 'atom_1_mean_dist',
       'atom_1_std_dist', 'nearby_H', 'mol_med_dist', 'atom_0_std_dist', 'x_1',
       'nearby_N', 'atom_1_kur_dist', 'mol_max_dist', 'mol_std_dist',
       'atom_1_min_dist', 'mol_mean_dist', 'y_0', 'mol_kur_dist',
       'atom_0_kur_dist', 'n_C', 'mol_min_dist', 'z_0', 'fc_pred', 'sd_pred',
       'pso_pred', 'dso_pred', 'mulliken_charge_pred']
meta_feats = ['fc_pred', 'sd_pred', 'pso_pred', 'dso_pred', 'mulliken_charge_pred']
meta_orig = ['fc', 'sd', 'pso', 'dso', 'mulliken_charge']
categoricals = [
    'type',
    'atom_1'
]
feats = list(set(feats) - set(categoricals) - set(meta_feats))
```

In [96]:
df.columns

Index(['molecule_name', 'atom_index_0', 'atom_index_1',
       'scalar_coupling_constant', 'fc', 'sd', 'pso', 'dso', 'mulliken_charge',
       'dip_x', 'dip_y', 'dip_z', 'potential_energy', 'XX', 'YX', 'ZX', 'XY',
       'YY', 'ZY', 'XZ', 'YZ', 'ZZ', 'type_1JHC', 'type_1JHN', 'type_2JHC',
       'type_2JHH', 'type_2JHN', 'type_3JHC', 'type_3JHH', 'type_3JHN'],
      dtype='object')

In [134]:
# Looking at Initial DataFrame For Important Features for Level 1 Models
feats = [
    'fc', 'sd', 'pso', 'dso',
#     'mulliken_charge',
#     'dip_x', 'dip_y', 'dip_z',
#     'potential_energy',
#     'XX', 'YX', 'ZX', 'XY', 'YY', 'ZY', 'XZ', 'YZ', 'ZZ',
    'type_1JHC', 'type_1JHN', 'type_2JHC', 'type_2JHH', 'type_2JHN', 'type_3JHC', 'type_3JHH', 'type_3JHN'
]

target = ['scalar_coupling_constant']
grouping = ['molecule_name']

In [133]:
model = RandomForestRegressor(n_jobs=-1)
gc.collect()

4389

In [114]:
gss = GroupShuffleSplit(n_splits=5)
X, y = df, df[target]

In [115]:
scores = cross_val_score(model, X[feats], y, scoring='neg_mean_absolute_error', 
                         groups=X['molecule_name'], cv=gss, n_jobs=-1)

In [101]:
print(np.mean(np.log(-scores))) # base + types

-3.603422944068521


In [118]:
# Try Linear Model
from sklearn.linear_model import LinearRegression

In [135]:
lmodel = LinearRegression(n_jobs=-1)

In [136]:
lmodel.fit(X[feats], y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)

In [137]:
gss = GroupShuffleSplit(n_splits=5)
scores = cross_val_score(model, X[feats], y, scoring='neg_mean_absolute_error', 
                         groups=X['molecule_name'], cv=gss, n_jobs=-1)
print(np.mean(np.log(-scores)))

-3.6002255716104563


In [140]:
# Stacking linear and rf
train_idxs, test_idxs = next(gss.split(X[feats], y, groups=df['molecule_name']))
X_train, X_test, y_train, y_test = X.iloc[train_idxs], X.iloc[test_idxs], y.iloc[train_idxs], y.iloc[test_idxs]
lmodel.fit(X_train[feats], y_train)
model.fit(X_train[feats], y_train)
y_pred_rf = model.predict(X_test[feats])
y_pred_lin = lmodel.predict(X_test[feats])

In [167]:
# Stacking linear and rf
final_pred = np.mean(np.concatenate([np.expand_dims(y_pred_rf, axis=1), y_pred_lin], axis=1), axis=1)

In [170]:
np.log(mean_absolute_error(y_test, final_pred))

-4.147273343609824

In [170]:
np.log(mean_absolute_error(y_test, final_pred))

-4.147273343609824